In [1]:
import pandas as pd
import numpy as np

In [5]:
def setup(test_file, answer_output_file, function, test=False):
    with open(test_file) as f:
        lines = f.readlines()
    with open(answer_output_file, 'r+') as a:
        test_count = int(lines[0])
        index = 1
        for i in range(test_count):
            M, K, N = lines[index].split(" ")
            Ms = np.array([ np.int32( 1e6 * float(m)) for m in lines[index+1].strip().split(" ")])
            As = np.array([ np.int32( 1e6 * float(a)) for a in lines[index+2].strip().split(" ")])
            Ss = np.array([ np.int32( 1e6 * float(s)) for s in lines[index+3].strip().split(" ")])
            assert(len(Ms) == int(M))
            assert(len(As) == int(K))
            assert(len(Ss) == int(N))
            answers = function(Ss, Ms, As)
            if not test:
                a.write("\n".join(answers) + "\n")
            index += 4
            print(i)
        if test:
            test_answer = "".join(a.readlines())
            assert "\n".join(answers) == test_answer

def stringify_duple(tup):
    return " ".join([str(e + 1) for e in tup])

def minimize_all_deltas(Ss, Ms, As):
    answers_dict = {}
    answers = []
    Mss = pd.Series(Ms).drop_duplicates()
    Ass = pd.Series(As).drop_duplicates()
    x, y = np.meshgrid(Mss, Ass)
    print("Dups M:", (len(Ms) - len(Mss)) / len(Ms))
    print("Dups A:", (len(As) - len(Ass))/ len(As))
    print("Dups S:", (len(Ss) - len(pd.Series(Ss).drop_duplicates()))/ len(Ss))

    X = pd.DataFrame((x+y), columns=Mss.index, index=Ass.index).T.stack()
    X = X[X>0]

    # X = (x + y).T
    # X[X <= 0] = 1e9
    print(X.dtype)
    for s in Ss:
        if s not in answers_dict.keys():
            a = abs(X - s)
            vals = a.idxmin()
            # i,j = np.unravel_index(np.argmin(a, axis=None), a.shape)
            # vals = (Mss.index[i], Ass.index[j])
            answers_dict[s] = stringify_duple(vals)
        answers.append(answers_dict[s])
    return answers

In [6]:
# %%timeit
ts = [1]
tMs = np.array([1, 3])
tAs = np.array([1, 0])
assert minimize_all_deltas(ts, tMs, tAs) == ['1 2']
# ts = 1
# tMs = np.array([1])
# tAs = np.array([-2, -3])
# # assert minimize_delta(ts, tMs, tAs) is None
ts = [2]
tMs = np.append([1], np.full(1000, 2))
tAs = np.append([1], np.full(1000, 2))
# will be slow without early stopping
minimize_all_deltas(ts, tMs, tAs)

setup('./all2/1.txt', './answer1.txt', minimize_all_deltas, test=True)

Dups M: 0.0
Dups A: 0.0
Dups S: 0.0
int64
Dups M: 0.998001998001998
Dups A: 0.998001998001998
Dups S: 0.0
int64
Dups M: 0.0
Dups A: 0.0
Dups S: 0.125
int32
0


In [7]:
%prun setup('./all2/2.txt', './answer2.txt', minimize_all_deltas, test=False)

Dups M: 0.7954070981210856
Dups A: 0.6198347107438017
Dups S: 0.7847826086956522
int32
0
Dups M: 0.0
Dups A: 0.0
Dups S: 0.0
int32
1
 

         484192 function calls (480578 primitive calls) in 0.703 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      567    0.056    0.000    0.056    0.000 {method 'argmin' of 'numpy.ndarray' objects}
      573    0.052    0.000    0.224    0.000 arraylike.py:231(array_ufunc)
    90584    0.038    0.000    0.062    0.000 {built-in method builtins.isinstance}
      567    0.037    0.000    0.037    0.000 {built-in method _operator.sub}
      571    0.017    0.000    0.028    0.000 multi.py:2022(__getitem__)
    37004    0.017    0.000    0.023    0.000 generic.py:30(_check)
     1168    0.015    0.000    0.260    0.000 series.py:238(__init__)
       34    0.014    0.000    0.014    0.000 socket.py:438(send)
        2    0.012    0.006    0.697    0.349 <ipython-input-5-74a22b4e547f>:27(minimize_all_deltas)
     1160    0.011    0.000    0.119    0.000 construction.py:423(sanitize_array)
    51587    0.010    0.000    0.01

In [8]:
%prun setup('./all2/3.txt', './answer3.txt', minimize_all_deltas, test=False)

Dups M: 0.8996990972918756
Dups A: 0.8016032064128257
Dups S: 0.9989916407014148
int32
0
Dups M: 0.0
Dups A: 0.0
Dups S: 7.016699745394037e-05
int32


KeyboardInterrupt: 

In [9]:
setup('./all2/4.txt', './answer4.txt', minimize_all_deltas, test=False)

Dups M: 0.9998997471623434
Dups A: 0.7995991983967936
Dups S: 0.8991935483870968
int32
0
Dups M: 0.000517682823644387
Dups A: 0.0
Dups S: 0.0


KeyboardInterrupt: 